<a href="https://colab.research.google.com/github/chrisluo5311/LangChain_Introduction/blob/main/%E6%9C%80%E5%A4%A7%E5%8C%96%E6%B4%BB%E7%94%A8%E5%A4%A7%E5%9E%8B%E8%AA%9E%E8%A8%80%E6%A8%A1%E5%9E%8B_Ch4_%E4%BD%BF%E7%94%A8_QLora_%E9%87%8F%E5%8C%96%E6%8A%80%E8%A1%93%E5%BE%AE%E8%AA%BF%E5%A4%A7%E8%AA%9E%E8%A8%80%E6%A8%A1%E5%9E%8B%EF%BC%8C%E7%B5%90%E5%90%88_bitandbytes_lib_%E8%88%87_PEFT_lib_%E6%95%B4%E5%90%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

開頭介紹詞語

可能有需要將讀取 Lora 模型＋原始模型隔離出去

# 安裝套件包

In [3]:
# Install latest bitsandbytes & transformers, accelerate from source
!pip install -qqq -U bitsandbytes
!pip install -qqq -U git+https://github.com/huggingface/transformers.git
!pip install -qqq -U git+https://github.com/huggingface/peft.git
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git
!pip install -qqq -U trl
# Other requirements for the demo
!pip install -qqq -U einops
!pip install -qqq -U datasets
!pip install -qqq -U wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

## 確認資源

In [6]:
!nvidia-smi

Sat Aug 26 05:23:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# LoRa 訓練

參考資料：[bnb-4-bit-training](https://colab.research.google.com/drive/1VoYNfYDKcKRQRor98Zbf2-9VQTtGJ24k?usp=sharing#scrollTo=Ybeyl20n3dYH)

## 讀取模型 in 4 bits

* load_in_4bit：模型將以4位精度載入到記憶體中。
* bnb_4bit_use_double_quant：QLoRa提出的雙量化。
* bnb_4bit_quant_type：這是量化的類型。 “nf4”代表4位的NormalFloat。
* bnb_4bit_compute_dtype：當以4位載入和存儲模型時，在需要時對其進行部分量化，並以16位精度（bfloat16）進行所有計算。


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "ybelkada/falcon-7b-sharded-bf16"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"":0},
    trust_remote_code=True
)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

## 查看模型架構

In [8]:
print(model)

RWForCausalLM(
  (transformer): RWModel(
    (word_embeddings): Embedding(65024, 4544)
    (h): ModuleList(
      (0-31): 32 x DecoderLayer(
        (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
        (self_attention): Attention(
          (maybe_rotary): RotaryEmbedding()
          (query_key_value): Linear4bit(in_features=4544, out_features=4672, bias=False)
          (dense): Linear4bit(in_features=4544, out_features=4544, bias=False)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): MLP(
          (dense_h_to_4h): Linear4bit(in_features=4544, out_features=18176, bias=False)
          (act): GELU(approximate='none')
          (dense_4h_to_h): Linear4bit(in_features=18176, out_features=4544, bias=False)
        )
      )
    )
    (ln_f): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4544, out_features=65024, bias=False)
)


## 預處理讀取模型

請使用 PEFT 當中 prepare_model_for_kbit_training 方法
，同時也請使用 model.gradient_checkpointing_enable()，這將會啟用梯度檢查點，這樣可以減少記憶體佔用，但是速度會稍微降低一些。

Github 參考文件[prepare_model_for_kbit_training](https://github.com/huggingface/peft/blob/08cb3dde577747f6ca6638c884fd66fd16cf2e9d/src/peft/utils/other.py#L62)

In [9]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


## 替模型準備 QLoRa 方法，每一層添加 adapter

根據 QLoRA論文 ，重要的是要考慮 Transformer lib 中的所有線性層，以獲得最佳性能。因此，除了 添加 `query key value` layer之外，我們還將在 target module 中添加 `dense` 、 `dense_h_to_4_h` 和 `dense_4h_to_h` 層。

In [10]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

model = get_peft_model(model, peft_config)

## 查看可以訓練的參數量

In [11]:
model.print_trainable_parameters()

trainable params: 130,547,712 || all params: 7,052,268,416 || trainable%: 1.8511449692387885


## 請多留意！PEFT Config 可以有多種策略


###參數一、target_mosildes:模型架構不同，Target_modules 不同

參考文件:[TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING](https://github.com/huggingface/peft/blob/08cb3dde577747f6ca6638c884fd66fd16cf2e9d/src/peft/utils/other.py#L220)

```
TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING = {
    "t5": ["q", "v"],
    "mt5": ["q", "v"],
    "bart": ["q_proj", "v_proj"],
    "gpt2": ["c_attn"],
    "bloom": ["query_key_value"],
    "blip-2": ["q", "v", "q_proj", "v_proj"],
    "opt": ["q_proj", "v_proj"],
    "gptj": ["q_proj", "v_proj"],
    "gpt_neox": ["query_key_value"],
    "gpt_neo": ["q_proj", "v_proj"],
    "bert": ["query", "value"],
    "roberta": ["query", "value"],
    "xlm-roberta": ["query", "value"],
    "electra": ["query", "value"],
    "deberta-v2": ["query_proj", "value_proj"],
    "deberta": ["in_proj"],
    "layoutlm": ["query", "value"],
    "llama": ["q_proj", "v_proj"],
    "chatglm": ["query_key_value"],
    "gpt_bigcode": ["c_attn"],
    "mpt": ["Wqkv"],
}
```

### 參數二、PEFT_TYPE

參考文件：[PEFT_TYPE_TO_CONFIG_MAPPING](https://github.com/huggingface/peft/blob/08cb3dde577747f6ca6638c884fd66fd16cf2e9d/src/peft/mapping.py#LL43C1-L43C10)

```
PEFT_TYPE_TO_CONFIG_MAPPING = {
    "ADAPTION_PROMPT": AdaptionPromptConfig,
    "PROMPT_TUNING": PromptTuningConfig,
    "PREFIX_TUNING": PrefixTuningConfig,
    "P_TUNING": PromptEncoderConfig,
    "LORA": LoraConfig,
    "ADALORA": AdaLoraConfig,
}
```

### 參數三、MODEL_TYPE

參考文件：[MODEL_TYPE](https://github.com/huggingface/peft/blob/08cb3dde577747f6ca6638c884fd66fd16cf2e9d/src/peft/mapping.py#LL35C1-L41C2)

```
MODEL_TYPE_TO_PEFT_MODEL_MAPPING = {
    "SEQ_CLS": PeftModelForSequenceClassification,
    "SEQ_2_SEQ_LM": PeftModelForSeq2SeqLM,
    "CAUSAL_LM": PeftModelForCausalLM,
    "TOKEN_CLS": PeftModelForTokenClassification,
    "QUESTION_ANS": PeftModelForQuestionAnswering,
}
```

## 讀取資料集

讓我們載入一個通用數據集，English_quote，以根據著名引號微調我們的模型。

參考文章:[Falcon lands in HF System](https://colab.research.google.com/drive/1qpvaDyxMosvElF2hKulAvs0s_rq17FE1?authuser=3#scrollTo=rS_J9gwa0swB)

In [12]:
from datasets import load_dataset

dataset_name = "Abirate/english_quotes"
dataset = load_dataset(dataset_name, split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [13]:
dataset

Dataset({
    features: ['quote', 'author', 'tags'],
    num_rows: 2508
})

## 載入 Trainer

這裡我們使用 TRL lib 中的 SFTTrainer ，該庫提供了圍繞變壓器 Trainer 的包裝器，以使用 PEFT adapter輕鬆 fine-tune基於指令的數據集上的模型。讓我們首先載入下面的訓練參數。

參考資料：[Falcon lands in HF system](https://huggingface.co/blog/zh/falcon)

In [14]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_8bit" #QLoRA 策略
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 30
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

## 使用 Supervised-FineTune Trainer;SFTTrainer

官方文件：[trl](https://huggingface.co/docs/trl/main/en/sft_trainer)

In [15]:
from trl import SFTTrainer
import transformers

max_seq_length = 512

# Require for Falcon
tokenizer.pad_token = tokenizer.eos_token

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    dataset_text_field="quote",
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [16]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

## 訓練模型

現在我們開始訓練 adapter - 微調

In [17]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

trainer.train()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,1.829100
20,1.236200
30,1.154800


TrainOutput(global_step=30, training_loss=1.4067166646321614, metrics={'train_runtime': 255.6274, 'train_samples_per_second': 1.878, 'train_steps_per_second': 0.117, 'total_flos': 489215789411328.0, 'train_loss': 1.4067166646321614, 'epoch': 0.19})

### 如果訓練失敗

出現資訊為：RuntimeError: unscale_() has already been called on this optimizer since the last update().

參考 [github issue @](https://github.com/huggingface/transformers/issues/23935#issuecomment-1588134562)

In [ ]:
!pip uninstall transformers
!pip install -U git+https://github.com/huggingface/transformers@de9255de27abfcae4a1f816b904915f0b1e23cd9

Found existing installation: transformers 4.31.0.dev0
Uninstalling transformers-4.31.0.dev0:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.31.0.dev0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled transformers-4.31.0.dev0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers (to revision de9255de27abfcae4a1f816b904915f0b1e23cd9) to /tmp/pip-req-build-f5eiaf69
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-f5eiaf69
  Running command git rev-parse -q --verify 'sha^de9255de27abfcae4a1f816b904915f0b1e23cd9'
  Running command git fetch -q https://github.com/huggingface/transformers de9255de27abfcae4a1f816b904915f0b1e23cd9
  Running command git checkout -q de9255de27abfcae4a1f816b904915f0b1e23cd

## 保存模型的方法

* 方法ㄧ、存放至 local 端
* 方法二、存放至 HuggingFace Hub 雲端

## 方法ㄧ、存放至 Local 端

模型當前是從 PEFT 拿到手得，所以保存上也只會保存我們訓練的 adapter 層級，也就是會超小！省空間！



In [ ]:
save_adapter_path="output_dir"
model.save_pretrained(save_adapter_path)

我的大概這種大小


![adapter_model.bin](https://i.imgur.com/6DocZpN.png)

##方法二、分享 LoRa 模型到 Hub 上

In [15]:
from huggingface_hub import notebook_login

notebook_login()

In [19]:
model.push_to_hub("chrisluo5311/falcon-7b-sharded-bf16-english-quote-qlora", use_auth_token=True, create_pr=1)

adapter_model.bin:   0%|          | 0.00/522M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/chrisluo5311/falcon-7b-sharded-bf16-english-quote-qlora/commit/034e26b9cf69595c5fa7ac2a8731889fd51095ea', commit_message='Upload model', commit_description='', oid='034e26b9cf69595c5fa7ac2a8731889fd51095ea', pr_url='https://huggingface.co/chrisluo5311/falcon-7b-sharded-bf16-english-quote-qlora/discussions/1', pr_revision='refs/pr/1', pr_num=1)

# 讀取 Lora 模型 + 原始模型

## 從 Hub 加載 adapters

You can also directly load adapters from the Hub using the commands below:

In [4]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "chrisluo5311/falcon-7b-sharded-bf16-english-quote-qlora"
peft_config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    return_dict=True,
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map='auto',
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

ValueError: ignored

## 進行推論


In [5]:
batch = tokenizer("Two things are infinite: ", return_tensors='pt', return_token_type_ids=False)

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

NameError: ignored